<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SST_DP2025/blob/main/Day_01/L03/L03_Exercise_Answer.ipynb)

# Lesson 03 Exercise

## User Registration and Order Processing System

You will build a basic user registration and order system using Python and Pydantic. The system will consist of:

1. `User Model`: Stores user details, including a validated email address.
2. `Order Model`: Contains the user's ID, product name, quantity, and price per item, with checks to ensure valid quantity and price values.
   
You will:
- Create Pydantic models for `User` and `Order` with field validations.
- Use Pydantic’s `EmailStr` type to automatically validate email addresses.
- Implement a method to calculate the total price of an order.
- Serialize the `Order` model as JSON.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U pydantic[email]

---

## Simple Project Code (Pydantic v2)

In [1]:
from pydantic import BaseModel, Field, EmailStr, field_validator, ConfigDict

In [2]:
# 1. User Model with Email Validation
class User(BaseModel):
    model_config = ConfigDict(from_attributes = True)
    id: int
    name: str = Field(min_length = 3, max_length = 50)

    # Automatically validates email format
    email: EmailStr  

In [3]:
# 2. Order Model with Simple Quantity and Price Validation
class Order(BaseModel):
    # from_attributes=True allows a Pydantic model to be initialized from objects or dictionary-like structures 
    # that have attributes, rather than requiring standard dictionaries
    model_config = ConfigDict(from_attributes = True)
    order_id: int
    user_id: int  # Links to the user placing the order
    product_name: str
    quantity: int = Field(gt = 0)  # Quantity must be positive
    price_per_item: float = Field(gt = 0)  # Price per item must be positive

    @field_validator("quantity", "price_per_item")
    def check_positive_values(cls, value):
        if value <= 0:
            raise ValueError("Quantity and price per item must be positive")
        return value

    def calculate_total_price(self):
        return self.quantity * self.price_per_item

In [4]:
# 3. Example Usage: Create User and Order

# Create a User with valid data
user = User(id=1, name = "Alice", email = "alice@example.com")

# Create an Order with valid data
try:
    order = Order(
        order_id = 101,
        user_id = user.id,
        product_name = "Keyboard",
        quantity = 2,
        price_per_item = 49.99
    )
    print("Order created successfully:")
    print(order.model_dump_json(indent = 2))  # Display order as JSON
    print(f"Total price for the order: ${order.calculate_total_price():.2f}")
except ValueError as e:
    print(f"Order creation failed: {e}")

Order created successfully:
{
  "order_id": 101,
  "user_id": 1,
  "product_name": "Keyboard",
  "quantity": 2,
  "price_per_item": 49.99
}
Total price for the order: $99.98


In [5]:
# Create an Order with invalid data
try:
    order = Order(
        order_id = 102,
        user_id = user.id,
        product_name = "Keyboard",
        quantity = 0,
        price_per_item = 49.99
    )
    print("Order created successfully:")
    print(order.model_dump_json(indent = 2))  # Display order as JSON
    print(f"Total price for the order: ${order.calculate_total_price():.2f}")
except ValueError as e:
    print(f"Order creation failed: {e}")

Order creation failed: 1 validation error for Order
quantity
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
